In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import numpy as np
import pandas as pd

BACKTESTING_MODULE_PATH = os.path.abspath('../../backtest')
sys.path.insert(1, BACKTESTING_MODULE_PATH)
from backtesting.backtesting import Backtest, Strategy

METHOD_MODULE_PATH = os.path.abspath('../..')
sys.path.insert(1, METHOD_MODULE_PATH)
import method.Ticker as Ticker
import method.SaleRules as _sr
import method.algofuncs as _af
path = os.getcwd()

Loading BokehJS ...

In [2]:
path = os.getcwd()
class FollowTheTrend(Strategy):
    def init(self):
        self.buy_price = 0
        self._index = 0
    def next(self):
        prices = self.data.Close
        opens = self.data.Open
        volumes = self.data.Volume
        if len(self.data.Volume) > 5:
            last_price = prices[-1]
            if self.buy_price == 0 and Ticker.isFollowTrendingV2(prices, volumes, opens[-1], 2.5):
                self.buy_price = last_price
                self.buy()
            if self.buy_price != 0 and (_sr.takeProfitByPercent(5, last_price, self.buy_price) or _sr.shouldCutLossByPercent(8, last_price, self.buy_price)):
                self.position.close()
                self.buy_price = 0

In [5]:
DATA_PATH = os.path.abspath('../../vn-stock-data/VNX/')
vnx_file = os.path.abspath('../../vn-stock-data/VNX.csv')
ticker_follow_trending = []
hose_ticker = _af.getHOSETickers(vnx_file)
reports = [['Ticker', 'Avg. Trade [%]', 'Equity Final [$]', 'Return [%]', 'Sharpe Ratio', 'Win Rate [%]', 'Trades']]
for ticker_id in hose_ticker:
    ticker = _af.get_pricing_by_path(DATA_PATH + '/' + ticker_id+'.csv', '2018-01-01', '2020-10-05')
    bt = Backtest(ticker, FollowTheTrend, commission=.005, exclusive_orders=False)
    stats = bt.run()
    if stats['_trades'].shape[0]:
        reports = np.append(reports, [[ticker_id, stats['Avg. Trade [%]'], stats['Equity Final [$]'], stats['Return [%]'], stats['Sharpe Ratio'], stats['Win Rate [%]'], stats['_trades'].shape[0]]], axis=0)
ls = pd.DataFrame(data=reports[1:,1:],index=reports[1:,0],columns=reports[0,1:])
ls_s = ls.sort_values(by=['Avg. Trade [%]'], ascending=False, na_position='first')
ls_s

,Avg. Trade [%],Equity Final [$],Return [%],Sharpe Ratio,Win Rate [%],Trades
VOS,8.97891494906422,10897.775200000002,8.977752000000018,0.5541202899024557,100.0,1
HAP,8.435879717706474,11758.112700000005,17.581127000000052,0.9186756103441461,100.0,2
SAM,8.417256331941815,10841.47365,8.414736499999998,0.6462578874287608,100.0,1
TVB,7.0573502816716704,11460.439790500002,14.604397905000024,0.5693260602185302,100.0,2
BFC,6.983877604458932,10698.258000000002,6.982580000000016,0.6631187128864224,100.0,1
...,...,...,...,...,...,...
SCR,-0.5061661049871269,9409.535471000012,-5.9046452899998805,0.0,58.333333333333336,12
HID,-0.4975124378109319,9950.262086,-0.49737913999999367,0.0,0.0,1
DCM,-0.4644441451742809,9829.763904500009,-1.7023609549999128,0.0,50.0,8
MBB,-0.30003688011063234,9793.439406000007,-2.0656059399999323,0.0,71.42857142857143,7
